In [ ]:
import pyPamtra
import numpy as np
import matplotlib.pyplot as plt
import sys
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import scipy
import random
import pandas as pd
from scipy.interpolate import UnivariateSpline
import glob
import time
import copy
import os
from tqdm import tqdm


In [ ]:
df_psd = pd.read_feather('./data/original_dataset.feather')   
params_from_dataset_list = pd.read_feather('./data/params_from_dataset_10spec.feather')
edr = 1e-4
save_path = './data/target_edr1e-4_10spec'

target_df = pd.DataFrame({'Ze_X': [], 'Ze_Ka':[], 'Ze_W':[], 
                                       'mdv_X':[], 'mdv_Ka':[], 'mdv_W':[],
                                       'sw_X':[], 'sw_Ka':[], 'sw_W':[],
                                       'sk_X':[], 'sk_Ka':[], 'sk_W':[],
                                       'spectra_X': [], 'spectra_Ka':[], 'spectra_W':[]})

In [10]:

########### DEFINE ENVIRONMENTAL PARAMETERS ###################
rho_air = 1.2


order_params = {
    'M0':0,
    'Deff':1,
    'mu':2,
    'a_mass_size':3,
    'b_mass_size':4,
    'alpha_area_size':5,
    'beta_area_size':6,
    
}

# Change ssrg coefs if necessary
# ssrg_coefs = [0.22,2.52,2.36,0.049]
# ssrg_coefs = [0.15,4.98,3.53,0.036]
# ssrg_coefs = [0.09,0.86,2.0,0.28]
# ssrg_coefs = [0.16,0.15,2.33,0.22]

# Define radial wind
wind_bs = 0. #-1.46 #0.73
# wind_w = {'W':wind_bs, 'Ka':wind_bs, 'X':wind_bs} # no dw
# wind_w = {'W':wind_bs, 'Ka':wind_bs+0.0747, 'X':wind_bs-0.0227} #dw0.1
wind_w = {'W':wind_bs, 'Ka':wind_bs-0.1167, 'X':wind_bs+0.2338} #dw0.25

for i in range(len(params_from_dataset_list)):
    params_from_dataset = params_from_dataset_list.iloc[i]

    theta = np.zeros(7)
    for k in order_params.keys():
        theta[order_params[k]] = params_from_dataset[k]


    ########### DEFINE MICROPHYSICAL PARAMETERS ##################

    Nt = theta[order_params['M0']] 
    Deff = theta[order_params['Deff']]

    aspect_ratio = 0.6
    canting_angle = 0.

    ######### RESHAPE PARAMETERS CORRECTLY ##################      

    pam = pyPamtra.pyPamtra()
    pam.df.addHydrometeor(('ice0', #name
                            aspect_ratio, #aspect ratio, <1 means oblate
                            -1, #phase: -1=ice, 1=liq
                            -99.,#200, #density
                            theta[order_params['a_mass_size']], #a parameter of mass-size
                            theta[order_params['b_mass_size']], #b parameter of mass-size
                            theta[order_params['alpha_area_size']], #alpha parameter of cross-section area - size relation
                            theta[order_params['beta_area_size']], #beta parameter of cross-section area - size relation
                            12, # moment provided in input file
                            200, #number of discrete size bins (internally, nbins+1 is used)
                            'mgamma', #name of psd
                            -99., #1st parameter of psd
                            -99.,#2nd parameter of psd
                            theta[order_params['mu']], #3rd parameter of psd
                            1., #4th parameter of psd
                            1e-5, #min diameter
                            1e-2, # max diameter
                            'ss-rayleigh-gans',#_%.3f_%.3f_%.3f_%.3f'%tuple(ssrg_coefs),
                            'heymsfield10_particles', 
                            canting_angle)) #canting angle of hydrometeors, only for Tmatrix and SSRG


    pam = pyPamtra.importer.createUsStandardProfile(pam,hgt_lev=np.array([np.linspace(999,1001,2).tolist()]))
    pam.p['temp_lev'][:]=params_from_dataset['temperature']+273.15
    pam.p['relhum_lev'][:]=90.
    pam.set["verbose"] = -1

    pam.p["hydro_n"][0,0,:,0] = theta[order_params['M0']]/rho_air
    pam.p["hydro_reff"][0,0,:,0] = 1/2*theta[order_params['Deff']]
        

    pam.nmlSet["radar_mode"] = "spectrum"
    pam.nmlSet['save_psd']=True
    pam.nmlSet["radar_noise_distance_factor"] = 1.0
    pam.nmlSet["radar_save_noise_corrected_spectra"]=  False
    pam.nmlSet['passive'] = False
    pam.nmlSet['radar_airmotion'] = True
    pam.nmlSet['radar_aliasing_nyquist_interv'] = 1
    pam.nmlSet['obs_height'] = 0
    pam.nmlSet['hydro_adaptive_grid'] = False
    pam.nmlSet['radar_allow_negative_dD_dU'] = True

    pam.nmlSet['radar_nfft']= params_from_dataset['fft_len_W']
    pam.nmlSet['radar_max_v']= params_from_dataset['v_nyq_W']
    pam.nmlSet['radar_min_v']= -params_from_dataset['v_nyq_W']
    vel0 = np.linspace(-params_from_dataset['v_nyq_W'],params_from_dataset['v_nyq_W'],params_from_dataset['fft_len_W'],endpoint=False)
    dv0 = vel0[1]-vel0[0]
    pam.nmlSet['radar_pnoise0'] = 10*np.log10(np.array(params_from_dataset['noise_level_Ka']))+10*np.log10(dv0*params_from_dataset['fft_len_Ka'])


    pred = {}
    
    for freq in ['X','Ka','W']:
        pam.p["wind_w"][:] = wind_w[freq]
        pam.addSpectralBroadening(edr, 10, params_from_dataset['beamwidth_deg_%s'%freq], params_from_dataset['integration_time_%s'%freq], params_from_dataset['freq_%s'%freq], kolmogorov=0.5)
        pam.runPamtra(params_from_dataset['freq_%s'%freq])
        pred['spectra_%s'%freq]=[pam.r['radar_spectra'][0,0,0,0,0,:]-10*np.log10(dv0)]
        pred['moments_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,:].tolist() 
        pred['Ze_%s'%freq] = pam.r["Ze"][0,0,0,0,0,0]
        pred['mdv_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,0]
        pred['sw_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,1]
        pred['sk_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,2]


    target_df.loc[i]=pd.Series({'Ze_X': pred['Ze_X'], 'Ze_Ka':pred['Ze_Ka'], 'Ze_W':pred['Ze_W'],
                                                'mdv_X':pred['mdv_X'], 'mdv_Ka':pred['mdv_Ka'], 'mdv_W':pred['mdv_W'],
                                                'sw_X':pred['sw_X'], 'sw_Ka':pred['sw_Ka'], 'sw_W':pred['sw_W'],
                                                'sk_X':pred['sk_X'], 'sk_Ka':pred['sk_Ka'], 'sk_W':pred['sk_W'],
                                                'spectra_X': pred['spectra_X'][0], 'spectra_Ka':pred['spectra_Ka'][0], 'spectra_W':pred['spectra_W'][0]
                                                })


In [10]:
target_df.to_feather(save_path)